# **Yilin Yang - HW4 Part2**

Parameters:

i: number of products

j: box number

k: subscriber type

N(k): number of subscribers in type j

c(i,k): cost of product i for subscriber k

Decision variable:

x(ij): whether to include product i in box j

y(jk): whether to send box j to subscriber k

u(i,j,k) = x(ij)*y(jk): whether to include i in box j and send to subscriber k

Revenue of products: 8 * sum_i sum_j sum_k (N(k) * x(ij) * y(jk)) = 8 * sum_i sum_j sum_k (N(k) * u(i,j,k))

Cost of products: sum_i sum_j sum_k (N(k) * x(ij) * y(jk) * c(ik)) = sum_i sum_j sum_k (N(k) * u(i,j,k) * c(ik))

In [45]:
%pip install gurobipy

In [46]:
from gurobipy import Model, GRB, quicksum
import numpy as np
import pandas as pd

In [85]:
# define products
products = np.array(['Moisturizer A','Moisturizer B', 'Moisturizer C', 'Moisturizer D',
                     'Lipstick A', 'Lipstick B', 'Lipstick C', 'Lipstick D',
                     'Concealer A', 'Concealer B', 'Concealer C', 'Concealer D',
                     'Highlighter A', 'Highlighter B', 'Highlighter C', 'Highlighter D',
                     'Nailpolish A', 'Nailpolish B', 'Nailpolish C', 'Nailpolish D'])
nb_products = len(products) # i


# define boxes
boxes = np.array(['Box 1', 'Box 2', 'Box 3'])
nb_boxes = len(boxes) # j

# define type of subscribers
subscribers = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
nb_subscribers_type = len(subscribers) #k

# define number of subscribers
N = np.array([8264, 5248, 7676, 5497, 23140, 12723, 20746, 22761, 6922, 3966])


# define c(ik)
c = np.array([
    [100,15,15,15,15,15,15,15,15,15],
    [18,18,18,100,18,18,18,18,18,18],
    [11,11,11,11,11,11,11,100,11,11],
    [18,100,100,18,18,18,100,18,18,18],
    [10,10,10,10,10,100,10,10,10,100],
    [7,7,7,7,7,100,100,100,100,100],
    [100,20,20,20,20,20,20,100,20,100],
    [9,9,9,9,9,9,9,9,9,9],
    [24,24,24,24,24,24,24,24,24,100],
    [100,100,100,100,6,6,6,6,6,6],
    [14,14,14,14,14,14,14,14,14,14],
    [24,24,100,100,24,24,24,24,24,24],
    [100,19,19,19,19,19,19,19,19,19],
    [100,19,19,19,19,19,19,19,19,19],
    [5,5,100,5,5,100,100,5,5,100],
    [6,6,6,100,6,6,6,6,6,100],
    [27,27,100,27,27,27,27,27,27,27],
    [15,15,15,15,15,15,15,15,100,15],
    [6,100,6,6,6,100,100,6,6,100],
    [100,9,9,9,9,9,9,9,9,9]
])



In [71]:
# define model
m = Model('Aleva case')

In [72]:
# decision variables
x = m.addVars(nb_products, nb_boxes, vtype = GRB.BINARY, name = 'product inclusion')
y = m.addVars(nb_boxes, nb_subscribers_type, vtype = GRB.BINARY, name = 'box assignment')
u = m.addVars(nb_products, nb_boxes, nb_subscribers_type, vtype = GRB.BINARY, name = 'total plan')

In [73]:
# calculate the revenue
revenue = 8 * quicksum(quicksum(quicksum(u[i,j,k] * N[k] for k in range(nb_subscribers_type)) for j in range(nb_boxes)) for i in range(nb_products))
cost = quicksum(quicksum(quicksum(u[i,j,k] * N[k] * c[i,k] for k in range(nb_subscribers_type)) for j in range(nb_boxes)) for i in range(nb_products))

# define objective function
m.setObjective(revenue - cost, GRB.MAXIMIZE)

In [74]:
# add constraints

# (1) box item constraint
for j in range(nb_boxes):
  m.addConstr(quicksum(x[i,j] for i in range(nb_products)) >= 4, name = 'box upper limit')
  m.addConstr(quicksum(x[i,j] for i in range(nb_products)) <= 7, name = 'box lower limit')

# (2) assignment constraint
for k in range(nb_subscribers_type):
  m.addConstr(quicksum(y[j,k] for j in range(nb_boxes)) == 1, name = 'assignment constraint')


# (3) resource constraint
for i in range(nb_products):
  m.addConstr(quicksum(quicksum(N[k] * u[i,j,k] for k in range(nb_subscribers_type)) for j in range(nb_boxes)) <= 80000, name = 'resource limit')

# (4) linearity adjustment
for i in range(nb_products):
  for j in range(nb_boxes):
    for k in range(nb_subscribers_type):
      m.addConstr(u[i,j,k] >= x[i,j] + y[j,k] - 1, name = 'linearity 1')
      m.addConstr(u[i,j,k] <= x[i,j], name = 'linearity 2')
      m.addConstr(u[i,j,k] <= y[j,k], name = 'linearity 3')

In [75]:
# update and solve the model
m.update()
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1836 rows, 690 columns and 4950 nonzeros
Model fingerprint: 0xed654b23
Variable types: 0 continuous, 690 integer (690 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [4e+03, 2e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+04]
Found heuristic solution: objective -9688269.000
Presolve removed 1014 rows and 0 columns
Presolve time: 0.03s
Presolved: 822 rows, 690 columns, 2922 nonzeros
Variable types: 0 continuous, 690 integer (690 binary)
Found heuristic solution: objective -9636711.000

Root relaxation: objective 7.338410e+05, 350 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntI

In [76]:
print(f'The optimal objective value is {m.ObjVal:g}.')

The optimal objective value is 137867.


In [96]:
product_plan = np.array([[x[i,j].X for j in range(nb_boxes)] for i in range(nb_products)])

print ('The optimal solution for including products in each box is:')
for j in range (nb_boxes):
  print(f'Box {j+1} includes: {[products[index] for index, value in enumerate(product_plan[:,j]) if value == 1]}.')

The optimal solution for including products in each box is:
Box 1 includes: ['Lipstick A', 'Highlighter C', 'Highlighter D', 'Nailpolish C'].
Box 2 includes: ['Lipstick A', 'Lipstick B', 'Lipstick D', 'Nailpolish D'].
Box 3 includes: ['Moisturizer C', 'Lipstick D', 'Concealer B', 'Nailpolish D'].


In [98]:
assignment_plan = np.array([[y[j,k].X for k in range(nb_subscribers_type)] for j in range(nb_boxes)])

print('The optimal solution for assigning boxes to different types of subscribers is:')
for j in range(nb_boxes):
  print(f'Box {j+1} should be assigned to subscriber type: {[subscribers[index] for index, value in enumerate(assignment_plan[j,:]) if value ==1]}.')

The optimal solution for assigning boxes to different types of subscribers is:
Box 1 should be assigned to subscriber type: [1, 5, 8, 9].
Box 2 should be assigned to subscriber type: [2, 3, 4].
Box 3 should be assigned to subscriber type: [6, 7, 10].
